# Import Packages

In [2]:
import pandas as pd
import language_tool_python
import numpy as np
import re
from tqdm import tqdm
from loguru import logger
from fuzzywuzzy import fuzz
import os
# import streamlit as st
# from stqdm import stqdm
# from time import sleep
from gingerit.gingerit import GingerIt
import Caribe as cb

c:\Users\prati\anaconda3\envs\excelcheckapp\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
tqdm.pandas()

# Load Files

In [3]:
special_char = pd.read_csv('Special characters list.csv')
ground_truth = pd.read_csv('DataStore/compare_datasets/GroundTruth.csv')

In [43]:
ground_truth['Brand'].unique()

array(['Mush', 'Strauss', 'Yellow Chimes', 'mCaffeine', 'AADAR',
       'Curryit', 'Nutty Yogi'], dtype=object)

In [5]:
','.join(ground_truth[ground_truth['Brand']=='Strauss']['ASIN'].tolist())

'B07GKZZYNT,B075M9WHY5,B07N19MQDW,B07QFVW623,B014PIG72M,B07DHKH4XD,B07QGZPDLK,B07QGZN74S,B014PIKNR2,B08FB3LV8F,B07QJ53146,B07TLW2QBR,B08GFCTNQN,B07Y9QG7BY,B07D8QCRDX,B07PSJ7672,B075JCJYN6,B08FB6Q7DW,B07D8PF4ZW,B07DHKXFGF,B08FB74ZC9,B07YB8BDNT,B07D8RR888,B082KQDWK3,B08HN9XJT5,B07QKMK7SG,B07D8NRZD1,B071NPMVBR,B07D8QSSVD,B07QK72B8N,B08XZ5GGF3,B08HN79WLM,B072BJLV3R,B08QRX6FRD,B01B6MV69Y,B01B6MV4FK,B075JBR528,B07QGZPDLY,B08XZ8XQLQ,B07MYFQPXF,B08FXG432Y,B07QFVWB1Y,B08Y91HVMW,B08XZ8D4NC,B07QK66R32,B07QJ53564,B07QCT15S5,B08XZ7FZBB,B07YB8D4ZS,B07QGZN6WC,B07D8SNSHH,B09HKQBM7M,B07QFVW5WL,B07QFVVZ1N,B085TG12FS,B07QJ53F9D,B08XZD971N,B0725M1L76,B08XZB19DD'

In [13]:
scraped_df = pd.DataFrame()
for i in os.listdir('DataStore/compare_datasets'):
    if 'GroundTruth' not in i:
        bdf = pd.read_csv('DataStore/compare_datasets/'+i)
        bdf['ScrapedFor'] = np.array([i.split('.')[0].split('_')[1].title() if i!='data_yc.csv' else 'Yellow Chimes']*bdf.shape[0])
        scraped_df = pd.concat([scraped_df,bdf])

In [18]:
req_cols = ['ScrapedFor','brand','ASIN','title','description','bullets']

# Overall Comparison

In [15]:
scraped_df = scraped_df[req_cols].copy()

In [16]:
scraped_df['bullets3'] = scraped_df['bullets'].str.replace('.  , ','\n')
scraped_df['bullets3'] = scraped_df['bullets3'].str.strip()

C:\Users\prati\AppData\Local\Temp\ipykernel_19332\3142955393.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  scraped_df['bullets3'] = scraped_df['bullets'].str.replace('.  , ','\n')


In [17]:
ground_truth.columns

Index(['Sr. No.', 'Brand', 'Category', 'Product Name', 'ASIN', 'Product URL',
       'Sales Frequency', 'Title', 'Description', 'Bulletpoints(3)', 'Grade',
       'Spell check', 'Dimensions details', 'Sentence case', 'A+ content',
       'Grade2', 'Main image', 'Additional images', 'Infographics',
       'Lifestyles', 'Video', 'A+ Content3', 'Info/lifestyle Total',
       'Total Images', 'Info present', 'Lifestyle present',
       'Video present or not', 'A+ content4', 'Image quality',
       'Product information', 'Remark', 'Score', 'Grade5', 'Final Grade',
       'Amazon USA', 'Amazon UK', 'Amazon CA', 'Amazon MX', 'Amazon Germany',
       'Amazon France', 'Amazon Italy', 'Amazon Spain', 'Amazon Netherlands',
       'Amazon Belgium', 'Amazon Sweden ', 'Amazon Poland', 'Amazon Australia',
       'Amazon Japan', 'Amazon Singapore', 'Amazon UAE'],
      dtype='object')

In [18]:
ground_truth = ground_truth[['Brand','ASIN','Title','Description', 'Bulletpoints(3)','Grade','Spell check', 'Dimensions details', 'Sentence case', 'A+ content','Grade2']].copy()

In [19]:
compare_df = scraped_df.merge(ground_truth,how = 'inner',on=['ASIN'])
compare_df.fillna('NA',inplace=True)

In [70]:
compare_df['Brand'].value_counts()

Yellow Chimes    151
AADAR             38
Strauss           21
mCaffeine         16
Mush               5
Name: Brand, dtype: int64

In [76]:
compare_df[['ScrapedFor','brand']].value_counts()

ScrapedFor     brand                     
Yellow Chimes   Yellow Chimes                150
Aadar          NA                             38
Strauss        NA                             21
Mcaffeine      NA                             16
Mush           NA                              5
Yellow Chimes   Melbees by Yellow Chimes       1
dtype: int64

# Excel Checks

In [25]:
parser = GingerIt()

###########################################################################################################
# Helper Functions
###########################################################################################################
## Sentence Case
def sentence_case(brand_name,title,brand_present_title):
    first_word = title.strip().split(' ')[0]
#     print(brand_name,first_word,brand_present_title)
    if brand_present_title==1:
        if brand_name.strip()[0]==first_word[0]:
            res = 1
        else:
            res = 0
    else:
        if first_word[0].isupper():
            res = 1
        else:
            res = 0
#     print(res)
    return res

###########################################################################################################

def spellcheck(txt):
    with language_tool_python.LanguageToolPublicAPI('en-US') as model:
        res = model.check(txt)
        logger.info(res)
        if len(res)>0:
            flg = 0
        else:
            flg = 1
        return [flg,res]

# def runGinger(txt,my_tool):
#     logger.info(txt)
#     sentences = txt.split('\n')
#     logger.info('sentences are : {}'.format('sentences'))
#     flg = 1
#     corrections = []
#     for sentence in sentences:
#         if len(sentence)>280:
#             words = sentence.split(' ')
#             for i in range(len(words)-50):
#                 sen = ' '.join(words[i:i+50])
#                 parse_res = my_tool.parse(sen)
#                 if len(parse_res['corrections'])>0:
#                     # for corr in parse_res['corrections']:
#                     # if corr['definition']!='Accept p'
#                     flg = 0
#                 corrections.append(parse_res['corrections'])
#         else:
#             parse_res = my_tool.parse(sentence)
#             if len(parse_res['corrections'])>0:
#                 flg = 0
#             corrections.append(parse_res['corrections'])
#     return [flg,corrections]

def runGinger(txt,my_tool):
    # logger.info(txt)
    sentences = [x.strip().lstrip(', ') for x in txt.strip().split('\n')]
    # logger.info('sentences are : {}'.format(sentences))
    flg = 1
    corrections = []
    for sentence in sentences:
        if len(sentence)>600:
            order = "[+-]?\d+\.\d+"
            first_str = re.sub(order, '', sentence)
            lines = re.split( r'[?.!]',first_str)
            for i in lines:
                parse_res = my_tool.parse(i)
                if len(parse_res['corrections'])>0:
                    # for corr in parse_res['corrections']:
                    # if corr['definition']!='Accept p'
                    flg = 0
                corrections.append(parse_res['corrections'])
        else:
            parse_res = my_tool.parse(sentence)
            if len(parse_res['corrections'])>0:
                flg = 0
            corrections.append(parse_res['corrections'])
    return [flg,corrections]
###########################################################################################################
## Special Character Check
def special_char_check(x):
    if len(set(special_char['chars'])) - len(set(special_char['chars'].tolist())-set(x))>0:
        return 0
    else:
        return 1

###########################################################################################################
## Get complete Title Flag
def get_Title_flag(data):
    ## Brand Name Present
    bn_check = lambda x,y:1 if y.strip().lower().find(x.strip().lower())>=0 else 0

    # t1 = st.empty()
    # t1.caption('Brand Check: ')
    data['title_brand_present'] = data[['brand','title']].progress_apply(lambda x:bn_check(x.brand,x.title),axis = 1)

    ## Sentence Case
    # t2 = st.empty()
    # t2.caption('Sentence Case Check: ')
    data['title_sentence_case'] = data[['brand','title',"title_brand_present"]].progress_apply(lambda x:sentence_case(x.brand,x.title,x.title_brand_present),axis = 1)

    ## Spell Check
    # t3 = st.empty()
    # t3.caption('Spell and Grammar Check: ')
    # data['title_spellcheck_res'] = data['title'].progress_progress_apply(lambda x:spellcheck(x,gf))
    data[['title_spellcheck','title_Corrected_text']] =  pd.DataFrame(data['title'].progress_apply(lambda x: runGinger(x,parser)).tolist())
    data['final_title_check_flag'] = data[['title_brand_present','title_sentence_case','title_spellcheck']].product(axis = 1)
    return data['final_title_check_flag']

###########################################################################################################
## Get complete Description Flag
def get_Description_flag(data):
    ## Special Character Check
    # d1 = st.empty()
    # d1.caption('Special Character Check: ')
    data['description_special_chr_check'] = data['description'].progress_apply(lambda x:special_char_check(x))
    ## Characters Constrained
    # d2 = st.empty()
    # d2.caption('Number of characters <2000 Check: ')
    data['description_char_constrained_2000'] = data['description'].progress_apply(lambda x:1 if len(x.strip())<=2000 else 0)
    ## Multiline Check
    def multiline_check(first_str):
        order = "[+-]?\d+\.\d+"
        first_str = re.sub(order, '', first_str)
        lines = re.split( r'[?.!]',first_str)
        if len(lines)>1:
            return 1
        else:
            return 0
    
    # d3 = st.empty()
    # d3.caption('Mutiline Check: ')
    data['description_multiline_check'] = data['description'].progress_apply(lambda x:multiline_check(x))
    ## Spell Check 
    # data['description_spellcheck'] = data['description'].progress_progress_apply(lambda x:spellcheck(x,gf))
    # d4 = st.empty()
    # d4.caption('Spell Grammar Check: ')
    data[['description_spellcheck','description_Corrected_text']] =  pd.DataFrame(data['description'].progress_apply(lambda x: runGinger(x,parser)).tolist())
    ## Final Description check Flag
    data['final_description_check_flag'] = data[['description_special_chr_check','description_char_constrained_2000','description_multiline_check','description_spellcheck']].product(axis = 1)

    return data['final_description_check_flag']

###########################################################################################################

## Get complete BulletPoints Flag
def get_BulletPoints_flag(data):
    ## Special Character check
    # b1 = st.empty()
    # b1.caption('Special Character Check: ')
    data['bullets_special_chr_check'] = data['bullets'].progress_apply(lambda x:special_char_check(x))
    ## Number of bullet points check (atleast 3 points)
    # b2 = st.empty()
    # b2.caption('Atleast 3 bullet Points Check: ')
    data['bullets_number_check'] = data['bullets'].progress_apply(lambda x:1 if x.count('\n')>=2 else 0)
    ## Bullet Points start with capital letter check
    # b3 = st.empty()
    # b3.caption('Bullet Points start with capital letter Check: ')
    data['bullets_first_capital_check'] = data['bullets'].progress_apply(lambda x: int(''.join([s.strip().lstrip(', ')[0] for s in x.strip().split('\n')]).isupper()) )
    ## Spell Check
    # data['bullets_spellcheck'] = data['bullets'].progress_progress_apply(lambda x:spellcheck(x,gf))
    # b4 = st.empty()
    # b4.caption('Spell Grammar Check Check: ')
    data[['bullets_spellcheck','bullets_Corrected_text']] =  pd.DataFrame(data['bullets'].progress_apply(lambda x: runGinger(x,parser)).tolist())
    ## Final Bullet Points check Flag
    data['final_bullet_point_check_flag'] = data[['bullets_special_chr_check','bullets_number_check','bullets_first_capital_check','bullets_spellcheck']].product(axis = 1)

    return data['final_bullet_point_check_flag']

###########################################################################################################
## Get complete Spell Check Flag
def get_sum(lst):
    return sum(lst)

def get_SpellCheck_flag(data):
    # s1 = st.empty()
    # s1.caption('Overall Spell-Check: ')
    data['final_entire_spellcheck'] = data[['title_spellcheck','description_spellcheck','bullets_spellcheck']].progress_apply(lambda x:1 if get_sum([x.title_spellcheck,x.description_spellcheck,x.bullets_spellcheck])==3 else 0,axis = 1)
    return data['final_entire_spellcheck']

###########################################################################################################
## Get complete Dimension Check Flag
def qc_dim(unit,values):
#     metric = {'meter':['m','meter'],
#     'centimeter':['cm','cms','centimeter'],
#     'millimeter':['mm','millimeter'],
#     'inches':['inch','inches']
#     'litre':['l','lit','litre'],
#     'gram':['g','gm','gram'],
#     'kilogram':['kg','kgms','kilogram']}
#     metric = ['meter','centimeter','millimeter','kilometer','inches','foot']
    metric2 = ['m','cms','mm','km','inches','ft']
    metric_change = [1,0.01,0.001,1000,0.0254,0.0348]
    metric_unit= []
    logger.info('unit for qc {}\nvalue for qc {}'.format(unit,values))
    for u in unit:
        umetric_ratio = []
        for m in metric2:
            umetric_ratio.append(fuzz.ratio(u,m))
        metric_unit.append(metric2[np.array(umetric_ratio).argmax()])
    logger.info('metric_unit {}'.format(metric_unit))
    updated_values = [float(v)*metric_change[metric2.index(i)] for i,v in zip(metric_unit,values)]
    logger.info('updated values {}'.format(updated_values))
    if len(set(metric_unit))>1:
        qc_res = [0,updated_values]
    else:
        qc_res = [1,updated_values]
    
    return qc_res

def format_dim(dim):
    logger.info('initial dim {}'.format(dim))
    dim = dim.replace(' ','').lower()
    dim = dim.replace('x',' ')
    units = []
    for unit in re.finditer("[a-z]+",dim):
        units.append(unit.group())
    values = []
    for val in re.finditer("[0-9]+",dim):
        values.append(val.group())
    logger.info('unit is {} and values are {}'.format(units,values))
    return [units,values]

def check_values(value_list):
    logger.info('value_list for check values {}'.format(value_list))
    max_len = [len(l) for l in value_list]
    if len(set(max_len))>1:
        logger.info('The dimensions length do not match!!')
        return 0
    value_list = [sorted(l) for l in value_list]
    logger.info('after sort {}'.format(value_list))
    same_val_list = np.array(value_list).T.tolist()
    logger.info('transform for each dim {}'.format(same_val_list))
    res = 1
    for v in same_val_list:
        ratio_list = [v[i]/v[0] for i in range(len(v))]
        logger.info('ratio list for list {} {}'.format(v,ratio_list))
        res*=np.prod([1 if (i<=1.05 and i>=0.95) else 0 for i in ratio_list])
    logger.info('res {}'.format(res))
    return res

def get_dimensions(text):
    iters = re.finditer("(((\d+ ?[a-zA-Z]+ ?)[x,X] ?(\d+ ?[a-zA-Z]+ ?)[x,X] ?(\d+ ?[a-zA-Z]+ ?))|((\d+ ?[a-z]+ ?)[x,X] ?(\d+ ?[a-zA-Z]+ ?)))|(((\d+ ?)[x,X] ?(\d+ ?)[x,X] ?(\d+ ?[a-zA-Z]+))|((\d+ ?)[x,X] ?(\d+ ?[a-zA-Z]+)))",text)
    matched_strings = []
    for i in iters:
        matched_strings.append(i.group())
    logger.info('matched_strings {} and length is {}'.format(matched_strings,len(matched_strings)==0))
    if len(matched_strings)==0:
        return 0
    same_unit_in_dim = 1
    multi_units_value = []
    for dim in matched_strings:
        units, values = format_dim(dim)
        if len(units)==1:
            units = units*len(values)
        qc_res = qc_dim(units,values)
        same_unit_in_dim*=qc_res[0]
        multi_units_value.append(qc_res[1])
    # dimres = [1,same_unit_in_dim,check_values(multi_units_value)]
    return check_values(multi_units_value)
            
def get_Dimensions_flag(data):
    # dim1 = st.empty()
    # dim1.caption('Dimensions Check: ')
    data['complete_data'] = data['title']+data['description']+data['bullets']#+
    data['final_dimensionality_check'] = data['complete_data'].progress_apply(lambda x: get_dimensions(x))
    # logger.info(data['dimension_check_res'])
    # data[['final_dimensionality_check','same_unit_in_dim','dimensionality_inter_check']] = data['dimension_check_res'].str.split('__%__')
    # data.drop('dimension_check_res',axis = 1,inplace = True)
    return data['final_dimensionality_check']

###########################################################################################################
## Get complete Spell Check Flag
def get_SentenceCase_flag(data):
    data['final_sentence_case_check'] = data['title_sentence_case']
    return data['final_sentence_case_check']

##############################################################################################################
## Get all the flags
def QC_check1(data):
    # text1 = st.empty()
    logger.info('Title Check Started')
    # text1.write('Title Check Started')
    data['final_title_check_flag'] = get_Title_flag(data)
    logger.info('Title Check Completed!!!')
    # text1.write('Title Check Completed!!!')

    # text2 = st.empty()
    logger.info('Description Check Started')
    # text2.write('Description Check Started')
    data['final_description_check_flag'] = get_Description_flag(data)
    logger.info('Description Check Completed!!!')
    # text2.write('Description Check Completed!!!')

    # text3 = st.empty()
    logger.info('Bullet Points Check Started')
    # text3.write('Bullet Points Check Started')
    data['final_bullet_point_check_flag'] = get_BulletPoints_flag(data)
    logger.info('Bullet Points Check Completed!!!')
    # text3.write('Bullet Points Check Completed!!!')

    data['Grade1'] = data[['final_title_check_flag','final_description_check_flag','final_bullet_point_check_flag']].sum(axis = 1)
    data['Grade1'] = data['Grade1'].apply(lambda x: 'A' if x==3 else('B' if x>0 else 'C'))

    # text4 = st.empty()
    logger.info('Entile Spell Check Started')
    # text4.write('Entile Spell Check Started')
    data['final_entire_spellcheck'] = get_SpellCheck_flag(data.copy())
    logger.info('Entile Spell Check Completed!!!')
    # text4.write('Entile Spell Check Completed!!!')

    # text5 = st.empty()
    logger.info('Dimensionality Check Started')
    # text5.write('Dimensionality Check Started')
    # data[['final_dimensionality_check','same_unit_in_dim','dimensionality_inter_check']] = get_Dimensions_flag(data.copy())
    data['final_dimensionality_check']= get_Dimensions_flag(data.copy())
    logger.info('Dimensionality Check Completed!!!')
    # text5.write('Dimensionality Check Completed!!!')

    # text6 = st.empty()
    logger.info('Sentence Case Check Started')
    # text6.write('Sentence Case Check Started')
    data['final_sentence_case_check'] = get_SentenceCase_flag(data.copy())
    logger.info('Sentence Case Check Completed!!!')
    # text6.write('Sentence Case Check Completed!!!')

    data['Grade2'] = data[['final_entire_spellcheck','final_dimensionality_check','final_sentence_case_check']].sum(axis = 1)
    data['Grade2'] = data['Grade2'].apply(lambda x: 'A' if x==3 else('B' if x==2 else 'C'))

    return data

In [21]:
scraped_df.fillna('NA',inplace=True)

res_df = QC_check1(scraped_df)

2023-02-17 15:52:25.771 | INFO     | __main__:QC_check1:284 - Title Check Started
  0%|          | 3/1025 [00:03<20:03,  1.18s/it]


KeyboardInterrupt: 

## Mush comparison

In [29]:
mush_df = pd.read_csv('DataStore/compare_datasets/data_mush.csv')
mush_df['ScrapedFor'] = np.array(['Mush']*mush_df.shape[0])
mush_df = mush_df[req_cols]
mush_df.fillna('NA',inplace = True)

In [20]:
mush_df.shape

(63, 6)

In [37]:
gt_mush_df = ground_truth[ground_truth['Brand'].str.lower()=='mush']

### ASINS Comparison

In [46]:
mush_df_asins = mush_df['ASIN'].tolist()

#### Duplicate Check

In [52]:
print('len(set(mush_df_asins))==mush_df.shape[0] = {}\n {}=={}'.format(len(set(mush_df_asins))==mush_df.shape[0],len(set(mush_df_asins)),mush_df.shape[0]))

len(set(mush_df_asins))==mush_df.shape[0] = False
 62==63


In [ ]:
mush_df['ASIN'].value_counts()[mush_df['ASIN'].value_counts()>1]

B07CWBTBRK    2
Name: ASIN, dtype: int64

One duplicate ASIN = B07CWBTBRK    

#### Different ASINS  

In GT not in scraped data

In [61]:
print(set(gt_mush_df['ASIN'].tolist()) - set(mush_df_asins))

{'B09G36WZF3', 'B0819PW15B', 'B09F5SR1GY', 'B07CW9WNLP', 'B07NY36LQR', 'B08P4XF1VP', 'B07S6P8652', 'B09F5Z518M', 'B07S8Q85B4', 'B07CWB65DY', 'B09XN9LHQQ', 'B07P12ZDYF', 'B07P12ZVDX', 'B07SRVPJMX', 'B08K8Y4WF7', 'B0823GKN83', 'B07NY34DJ1', 'B09XN6J7DY', 'B07NY36KRZ', 'B07GCHY9HX', 'B08P5HD75Q', 'B09XN9T786', 'B07R6DDL5C', 'B09XN7X8L1', 'B07WD8JL8S', 'B07CWCGF26', 'B09XN7KHVF', 'B09F5XLT7G', 'B09XN7WD3Z', 'B07CWBTB4K', 'B08P5JGJ5D', 'B09F5PL8HR', 'B07P1381QZ', 'B09G37TPTR', 'B09XN86Y57', 'B07CWBDFSK', 'B08Y365QSF', 'B07SVFGHC1', 'B07CWBZWFJ', 'B08P4X9Y9B', 'B07STFB7XD', 'B07KKHC2VD', 'B07SK8JNT7'}


In scraped data not in GT

In [62]:
print(set(mush_df_asins) - set(gt_mush_df['ASIN'].tolist()))

{'B0BRKVVD5R', 'B09Y2G7ZFN', 'B0B51TRRSB', 'B07CW1BJQY', 'B0BC96YLS5', 'B07RCHPL8M', 'B0BNVZ6SMB', 'B0855VDLD9', 'B08561NXF1', 'B07SJ5KCMG', 'B09MDPJSB1', 'B07SJ4Y93D', 'B09S5PM92D', 'B07KXV7MQR', 'B09X48CMKB', 'B08WQ5R7KQ', 'B0BRN1NQD6', 'B09X48RT33', 'B0BRN3K7QV', 'B0BRCR6CN3', 'B0B2WVWJFM', 'B09YMC5B9F', 'B0BP7QW7KV', 'B0BNBVHKZW', 'B0BN664HLW', 'B0855W7KL4', 'B0BRN1JQ5M', 'B09X48T3VM', 'B0BRKV5498', 'B0BDVDF8DJ', 'B0BRKVGC61', 'B0BNNKFCBZ', 'B09X49HP4M', 'B0BRKV7N7W', 'B09W5NZMPJ', 'B0BLZSRMMN', 'B08CMNNHYC', 'B08VF35G1Q', 'B0BC99R3K8', 'B0BSTX12LK', 'B0BQCPFH3M', 'B0BRLBXMDD', 'B0BRN2S13L', 'B0BQ7CFK5P', 'B089T7Z74M', 'B0BRKV7R6J', 'B0BDV8JQZF', 'B0BF4ZRKF1', 'B09Z89H7DC', 'B0BPCYPFMZ', 'B08WPWB9MT', 'B0BM45JH9N', 'B09X48CK2Z', 'B0BPF1TW2L', 'B0728KV2ZL', 'B0BQ751Y1B', 'B08CN3VSKV', 'B09X48F423'}


In [1]:
ground_truth[ground_truth['Brand']=='Mush']

NameError: name 'ground_truth' is not defined

In [39]:

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("KES/T5-KES")


In [40]:
model = AutoModelForSeq2SeqLM.from_pretrained("KES/T5-KES")

OSError: Unable to load weights from pytorch checkpoint file for 'C:\Users\prati/.cache\huggingface\hub\models--KES--T5-KES\snapshots\fb5378c138b6d2827be5583787178b4e467d7b12\pytorch_model.bin' at 'C:\Users\prati/.cache\huggingface\hub\models--KES--T5-KES\snapshots\fb5378c138b6d2827be5583787178b4e467d7b12\pytorch_model.bin'. If you tried to load a PyTorch model from a TF 2.0 checkpoint, please set from_tf=True.

In [ ]:

text = "I am lived with my parenmts "
inputs = tokenizer("grammar:"+text, truncation=True, return_tensors='pt')

output = model.generate(inputs['input_ids'], num_beams=4, max_length=512, early_stopping=True)
correction=tokenizer.batch_decode(output, skip_special_tokens=True)
print("".join(correction)) #Correction: I am living with my parents.


In [38]:
correct = mush_df['bullets'].progress_apply(lambda x : cb.Parser(x).TT_Parser())

  2%|▏         | 1/63 [00:02<02:23,  2.32s/it]


OSError: Unable to load weights from pytorch checkpoint file for 'C:\Users\prati/.cache\huggingface\hub\models--KES--T5-TTParser\snapshots\24b137cc238c008d35cc0bb4a30b413bdc6d6cfd\pytorch_model.bin' at 'C:\Users\prati/.cache\huggingface\hub\models--KES--T5-TTParser\snapshots\24b137cc238c008d35cc0bb4a30b413bdc6d6cfd\pytorch_model.bin'. If you tried to load a PyTorch model from a TF 2.0 checkpoint, please set from_tf=True.

In [23]:
mush_df[['bullets_spellcheck','bullets_Corrected_text']] = mush_df['bullets'].progress_apply(lambda x:spellcheck(x,my_tool)).tolist()

  2%|▏         | 1/63 [00:04<04:27,  4.32s/it]


TypeError: list indices must be integers or slices, not str

In [36]:
mush_df['bullets_new'] = mush_df['bullets'].apply(lambda x: '\n'.join([s.strip() for s in x.strip().split('.  , ')]))

In [59]:
res = []
for sen in mush_df['bullets_new'][0].split('\n'):
    res.append(spellcheck(sen)[1])
    

2023-02-20 17:06:41.827 | INFO     | __main__:spellcheck:28 - [Match({'ruleId': 'MORFOLOGIK_RULE_EN_US', 'message': 'Possible spelling mistake found.', 'replacements': ['CMS', 'cm', 'CDs', 'CSS', 'CNS', 'RMS', 'SMS', 'CCS', 'CFS', 'CMA', 'CME', 'CMG', 'CMOS', 'CVS', 'DMS', 'VMS', 'cams', 'ms', 'BMS', 'CGS', 'CLS', 'CMB', 'CMF', 'CMU', 'MMS', 'PMS', 'JMS', 'PMs', 'ems', 'CBS', 'HMS', 'oms', 'AMS', 'CDS', 'CES', 'CHS', 'CJS', 'CKS', 'CM', 'CMC', 'CMD', 'CMI', 'CMJ', 'CMK', 'CML', 'CMM', 'CMO', 'CMP', 'CMR', 'CMSF', 'CMT', 'CMV', 'CMW', 'CMX', 'CPS', 'CQS', 'CRS', 'CS', 'CZS', 'Cm', 'Cs', 'FMS', 'FMs', 'GMS', 'IMS', 'KMS', 'LMS', 'MS', 'Ms', 'OMS', 'TMS', 'UMS', 'WMS', 'XMS', 'cis', 'cos', 'cps', 'cs', 'cums', 'ACMs', 'CAS', 'CCs', 'CEMS', 'CIS', 'CKs', 'CMOs', 'CPMs', 'CTs', 'CVs', 'DMs', 'IMs', 'LMs', 'VMs', 'cmd', 'cts', 'dms', 'pms', 'is', 'was', 'as', 'his', 'has', 'can', 'me', 'come', 'its', 'my', 'us', 'yes', 'came', 'km', 'arms', 'car', 'cars', 'case', 'cast', 'comes', 'cost', 'cu

In [66]:
len(res)

3

In [58]:
len(res1[1])

5